Install pytorch

In [ ]:
!pip3 install torch torchvision  

Install CUDA

In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!sudo mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget http://developer.download.nvidia.com/compute/cuda/11.0.2/local_installers/cuda-repo-ubuntu1804-11-0-local_11.0.2-450.51.05-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu1804-11-0-local_11.0.2-450.51.05-1_amd64.deb
!sudo apt-key add /var/cuda-repo-ubuntu1804-11-0-local/7fa2af80.pub
!sudo apt-get update
!sudo apt-get -y install cuda

--2020-08-31 11:10:10--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.39.144
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.39.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [application/octet-stream]
Saving to: ‘cuda-ubuntu1804.pin’

cuda-ubuntu1804.pin 100%[===================>]     190  --.-KB/s    in 0s      

2020-08-31 11:10:10 (4.33 MB/s) - ‘cuda-ubuntu1804.pin’ saved [190/190]

--2020-08-31 11:10:10--  http://developer.download.nvidia.com/compute/cuda/11.0.2/local_installers/cuda-repo-ubuntu1804-11-0-local_11.0.2-450.51.05-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.39.144
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.39.144|:80... connected.
HTTP request sent, awaiting response... 200 OK


In [ ]:
#Check CUDA version
!cat /usr/local/cuda/version.txt

CUDA Version 11.0.228


Import paths of modules and datasets for training

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive
import os
import sys
os.chdir("/content/drive/My Drive/IPC-SHM_Project1_LIN_YANG/Python_Codes")
!ls

/bin/bash: google-drive-ocamlfuse: command not found
cuda-repo-ubuntu1804-11-0-local_11.0.2-450.51.05-1_amd64.deb  README.md
drive							      train.ipynb
evaluate.ipynb						      Validation
Modules							      Verification
predict.ipynb


Start to train

In [ ]:
from Modules.dataset import DataLoader
from torch import optim
import torch.nn as nn
import torch

# -------------------------------------
# import propsed U-net models 
# ----------------------------------
from Modules.unet_model import UNet
# from Modules.unet_model_attn import UNet
# from Modules.unet_model_attn_ad import UNet
# from Modules.unet_model_attn_full import UNet

torch.cuda.empty_cache()

def train_net(net, device, data_path, epochs=25, batch_size=1, lr=1e-17):
    # Load training set
    isbi_dataset = DataLoader(data_path)
    train_loader = torch.utils.data.DataLoader(dataset=isbi_dataset,
                                               batch_size=batch_size,
                                               shuffle=True)

    # optimizer = optim.RMSprop(net.parameters(), lr=lr, weight_decay=1e-8, momentum=0.9)
    optimizer = optim.Adam(net.parameters(), lr=lr, betas=(0.9, 0.999), 
                           eps=1e-08, weight_decay=1e-8, amsgrad=False)
    # Define Loss algorithm
    criterion = nn.BCEWithLogitsLoss()
    # best_loss statistics, initialized to positive infinity
    best_loss = float('inf')

    # -------------------------------------
    # import pth files 
    # ! Please notice the right path !
    # ----------------------------------
    best_model = 'best_model.pth'

    # Training epochs times
    for epoch in range(epochs):
        # Training mode
        net.train()

        # Start training according to batch_size
        for image, label in train_loader:
            optimizer.zero_grad()

            # Copy data to device
            image = image.to(device=device, dtype=torch.float32)
            label = label.to(device=device, dtype=torch.float32)

            # Use network parameters to output prediction results
            pred = net(image)

            # Calculate loss
            loss = criterion(pred, label)
            print('Loss/train', loss.item())

            # Save the network parameters with the smallest loss value
            if loss < best_loss:
                best_loss = loss
                torch.save(net.state_dict(), best_model)

            # Update parameters
            loss.backward()
            optimizer.step()
        print("Epoch:", epoch)


if __name__ == "__main__":
    # Select the device, cuda or cpu
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load picture channel 1, set classes as 1。
    net = UNet(n_channels=1, n_classes=1)
    net.to(device=device)

    # Specify the training set path
    data_path = "Verification/dataS/train/"
    train_net(net, device, data_path)